This contains the Python scripts to interact with the SQL instance and extract data from the mini_net database created.

In [2]:
%pip install mysql.connector

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
import os

load_dotenv()


True

Establish the connection with the sql instance 

In [12]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

host = os.getenv('host')
print(host)
user = os.getenv('user')
password = os.getenv('password')
database = os.getenv('database')


connection = create_db_connection(host, user, password,database)

34.123.122.214
MySQL Database connection successful


Point to the mini_net database

In [13]:
cursor = connection.cursor()
cursor.execute("USE mini_net;")

List the tables within the data base

In [14]:

cursor.execute("SHOW TABLES;")

for (databases) in cursor:
     print(databases[0])

bytearray(b'Actor')
bytearray(b'Category')
bytearray(b'FavoriteList')
bytearray(b'FavoriteListVideo')
bytearray(b'Plan')
bytearray(b'Review')
bytearray(b'Subscription')
bytearray(b'Users')
bytearray(b'Video')
bytearray(b'VideoActor')
bytearray(b'WatchHistory')
bytearray(b'Watch_History')


List all users from the Users table

In [7]:
query = """
SELECT *
FROM Users;
"""

cursor.execute(query)
results = cursor.fetchall()

for result in results:
  print(result)

(2, 'john_doe', 'John', 'Doe', 'john.doe@example.com', 'hashed_password_1', 'New York', 'USA', 30, 1, datetime.datetime(2024, 7, 6, 23, 7, 12), datetime.datetime(2024, 7, 6, 23, 7, 12))
(3, 'alice_smith', 'Alice', 'Smith', 'alice.smith@example.com', 'hashed_password_2', 'London', 'UK', 25, 2, datetime.datetime(2024, 7, 6, 23, 7, 33), datetime.datetime(2024, 7, 6, 23, 7, 33))
(4, 'mike_jones', 'Mike', 'Jones', 'mike.jones@example.com', 'hashed_password_3', 'Los Angeles', 'USA', 35, 3, datetime.datetime(2024, 7, 6, 23, 8, 3), datetime.datetime(2024, 7, 6, 23, 8, 3))
(5, 'sara_brown', 'Sara', 'Brown', 'sara.brown@example.com', 'hashed_password_4', 'Sydney', 'Australia', 28, 4, datetime.datetime(2024, 7, 6, 23, 8, 21), datetime.datetime(2024, 7, 6, 23, 8, 21))
(6, 'mark_wilson', 'Mark', 'Wilson', 'mark.wilson@example.com', 'hashed_password_5', 'Toronto', 'Canada', 32, 5, datetime.datetime(2024, 7, 6, 23, 8, 44), datetime.datetime(2024, 7, 6, 23, 8, 44))
(9, 'johnY_Fary', 'Johny', 'fary', '

Get the city from user and look for details pertaining to the city

In [9]:
import pandas as pd


def get_actors_by_city():

    # SQL query to select data for the specified city using a parameterized query
    query = """
        SELECT City, count(ActorID), round(Avg(Age),2)
        FROM Actor 
        GROUP BY City
    """
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    result = cursor.fetchall()
    city_actors= pd.DataFrame(result)

    
    if not result:
        return pd.DataFrame(), pd.DataFrame()  # Return empty DataFrames if no result is found

   
    return city_actors



city_actors  = get_actors_by_city()

# Show the results
print(f"Actors from city:")
print(city_actors)

 

Actors from city:
          City  count(ActorID) round(Avg(Age),2)
0      Concord               1             68.00
1  Los Angeles               3             40.33
2     New York               3             36.00
3    Melbourne               1             40.00
4  Rosh HaAyin               1             39.00


Get the Genre from user and retrieve the details for the genre

In [15]:
import pandas as pd

# Function to get favorite genres for a user
def get_favorite_genres(Genre):
    # SQL query to select favorite genres for the specified user
    query = f"""
    SELECT U.username, V.VideoID, V.Title, V.ReleaseDate, C.CategoryName
    FROM Users as U
    LEFT JOIN FavoriteList as F ON U.UserID = F.UserID
    LEFT JOIN FavoriteListVideo as FV ON F.FavoriteListID = FV.FavoriteListID
    LEFT JOIN Video as V ON FV.VideoID = V.VideoID
    LEFT JOIN Category as C ON V.CategoryID = C.CategoryID
    WHERE C.CategoryName = '{Genre}'
    """
    
    # Execute the query and fetch data
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Load the data into a pandas DataFrame
    user_favorites = pd.DataFrame(result)
    
    return user_favorites

# Assuming the connection is already established and assigned to the variable `connection`
# Get user input for the username
Genre = input("Enter the Genre: ")

# Get favorite genres for the specified user
user_favorites = get_favorite_genres(Genre)

# Show the results
if user_favorites.empty:
    print(f"No favorite genres found for user '{Genre}'.")
else:
    print(f"Favorite genres for user '{Genre}':")
    print(user_favorites)

Favorite genres for user 'comedy':
      username  VideoID                     Title ReleaseDate CategoryName
0   mike_jones        8  Mystery of the Lost City  2021-10-10       comedy
1   sara_brown        7       The Great Adventure  2022-03-15       comedy
2  mark_wilson        3                  Superbad  2007-08-17       comedy


get the subscription count by country 

In [10]:
query = """
    SELECT Country, COUNT(UserID) as subscriptioncount
    FROM Users
    GROUP BY Country
    """

# Execute the query and fetch data
cursor = connection.cursor(dictionary=True)
cursor.execute(query)
result = cursor.fetchall()
    
    # Load the data into a pandas DataFrame
subscription_counts = pd.DataFrame(result)


if subscription_counts.empty:
        print("No subscription data found.")
else:
        print("Subscription count by country:")
        print(subscription_counts)

Subscription count by country:
     Country  subscriptioncount
0        USA                  2
1         UK                  3
2  Australia                  1
3     Canada                  1


Get the keyword from user and look for movies starting with that keyword

In [11]:
def get_movies_by_keyword(keyword):
    # SQL query to get movies starting with the specified keyword
    query = f"""
    SELECT V.VideoID, V.Title, C.CategoryName, V.ReleaseDate
    FROM Video as V
    LEFT JOIN Category as C ON V.CategoryID = C.CategoryID
    WHERE V.Title LIKE '{keyword}%'
    """
    
    # Execute the query and fetch data
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Load the data into a pandas DataFrame
    movies = pd.DataFrame(result)
    
    return movies


keyword = input("Enter the keyword to search for movies: ")

movies = get_movies_by_keyword(keyword)

if movies.empty:
    print(f"No movies found starting with '{keyword}'.")
else:
    print(f"Movies starting with '{keyword}':")
    print(movies)

Movies starting with 'the':
   VideoID                     Title CategoryName ReleaseDate
0        2  The Shawshank Redemption        drama  1994-09-23
1        6              The Notebook     romantic  2004-06-25
2        7       The Great Adventure       comedy  2022-03-15


In [12]:
query = """

SELECT  CategoryName, count(*) as subscriptioncount
FROM Watch_History as W
left join Video AS V
on W.VideoID = V.VideoID
LEFT JOIN Category as C
ON V.CategoryID = C.CategoryID
GROUP BY CategoryName
"""

cursor.execute(query)
results = cursor.fetchall()

category_count = pd.DataFrame(results)
print(category_count) 

  CategoryName  subscriptioncount
0       sci-fi                  1
1        drama                  1
2       comedy                  1


get the subscription type from user and get their details

In [13]:

def get_youngest_customer_details(subscription_type):
    try:
        cursor = connection.cursor(dictionary=True)
        query = """
            SELECT U.UserID, UserName, Concat(FirstName , ' ' , LastName) as Fullname , 
            Email, City, Country, PasswordHash, Age, SubscriptionType
            FROM Plan AS P
            LEFT JOIN Users AS U ON P.UserID = U.UserID
            LEFT JOIN Subscription AS S ON P.SubscriptionID = S.SubscriptionID
            WHERE S.SubscriptionType = %s
            ORDER BY U.Age ASC
            LIMIT 1
        """
        cursor.execute(query, (subscription_type,))
        #result = cursor.fetchone()
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"Error fetching data: {e}")
        return None

subscription_type = input("Enter the subscription category (e.g., 'UHD'): ")

    # Get youngest customer details in the specified subscription category
result = get_youngest_customer_details(subscription_type)

    # Display the result
if result:
        print(f"Youngest customer in {subscription_type} subscription category:")
        df = pd.DataFrame(result)
        print(df)
        
        # print(f"Username: {result['UserName']}")
        # print(f"City: {result['City']}")
else:
        print(f"No data found for subscription category '{subscription_type}'.")
    

 

Youngest customer in uhd subscription category:
   UserID     UserName     Fullname                    Email    City Country  \
0       3  alice_smith  Alice Smith  alice.smith@example.com  London      UK   

        PasswordHash  Age SubscriptionType  
0  hashed_password_2   25              UHD  


get age range and view the subscription details

In [14]:
def get_subscription_by_age(age_range1, age_range2):
    sql_query = """
    SELECT U.UserID, UserName, CONCAT(FirstName, ' ', LastName) as Fullname, Email, City, Country, PasswordHash, Age, SubscriptionType
    FROM Plan as P
    LEFT JOIN Users as U ON P.UserID = U.UserID
    LEFT JOIN Subscription as S ON P.SubscriptionID = S.SubscriptionID
    WHERE Age BETWEEN age_range1 AND age_range2
    """

       # Execute the query and fetch data
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Load the data into a pandas DataFrame
    subscription_age = pd.DataFrame(result)
    
    return subscription_age

    

lower_age_range = input("Enter the lower age range: ")
upper_age_range = input("Enter the upper age range: ")

subscription_age = get_subscription_by_age(lower_age_range, upper_age_range)

if subscription_age.empty:
    print(f"No subscription found for this age range between '{lower_age_range}' and '{upper_age_range }'.")
else:
    print(f"Subscription details for the age range between'{lower_age_range}' and '{upper_age_range}':")
    print(subscription_age)
 

Subscription details for the age range between'22' and '30':
  CategoryName  subscriptioncount
0       sci-fi                  1
1        drama                  1
2       comedy                  1
